<a href="https://colab.research.google.com/github/Ravi-agenc/AGenC/blob/main/RAG_chatbot_with_exceptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain_community.llms import OpenAI
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader
import uvicorn

In [ ]:

# Environment setup
os.environ['OPENAI_API_KEY'] = ''

# Initialize FastAPI app
app = FastAPI()


In [ ]:
#! Already in the prompt_model.py file
# Request model for user input
class PromptInput(BaseModel):
    prompt: str

In [ ]:
# Initialize the language model and memory
llm = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0, max_tokens=256)
memory_1 = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, verbose=True, memory=memory_1)

In [ ]:
# Load the data - change as needed
os.system('wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip')
os.system('unzip -q new_articles.zip -d new_articles')

In [ ]:
# Initialize document loader and process text files
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
# Create and persist vector database
persist_directory = 'db'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
# Create a retriever and QA chain
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)


In [ ]:
threshold = 5
conv_counter = 0


In [ ]:
def switch_memory_if_needed(conversation_chain, input_text, conv_counter):
    output = conversation_chain.predict(input=input_text)
    conversation_length = len(conversation_chain.memory.buffer)

    if conv_counter == threshold + 1:
        memory_2 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150, chat_memory=ChatMessageHistory(messages=memory_1.chat_memory.messages))
        conversation_chain.memory = memory_2
        print("Switched to ConversationSummaryBufferMemory")

    return output



In [ ]:
def process_llm_response(llm_response):
    if llm_response['result']:
        return llm_response['result']
    return None


In [ ]:
def prompt_llm(prompt_input: PromptInput):
    global conv_counter
    try:
        user_input = prompt_input.prompt
        conv_counter += 1

        # Use RAG to retrieve relevant documents and get the response
        llm_response = qa_chain(user_input)
        rag_response = process_llm_response(llm_response)

        # Generate a general response based on conversation context
        general_response = switch_memory_if_needed(conversation, user_input, conv_counter)

        if rag_response:
            return {"response": rag_response, "source": "RAG"}
        else:
            return {"response": general_response, "source": "General"}

    except ValueError as ve:
        raise HTTPException(status_code=400, detail=str(ve))

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

    except NotFound as nf:
        raise HTTPException(status_code=404, detail=str(nf)

